# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [ ]:
import gradio as gr
from openai import OpenAI
import os
import json
import requests
from bs4 import BeautifulSoup

OPENROUTER_BASE_URL = "https://openrouter.ai/api/v1"
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
if OPENROUTER_API_KEY and OPENROUTER_API_KEY.startswith("sk-or-"):
    print("OPENROUTER_API_KEY looks good so far")
else:
    print("OPENROUTER_API_KEY doesn't seem right")

openrouter = OpenAI(base_url=OPENROUTER_BASE_URL, api_key=OPENROUTER_API_KEY)

MODELS = [
    "openai/gpt-4o-mini",
    "openai/gpt-4o",
    "anthropic/claude-3.5-sonnet",
    "google/gemini-flash-1.5",
    "meta-llama/llama-3.1-8b-instruct:free",
]

# ── Tool implementations ─────────────────────────────────────────────────────

def get_pr_guide():
    """Scrape Ed Donner's PR guide and return plain text."""
    try:
        resp = requests.get("https://edwarddonner.com/pr", timeout=10)
        soup = BeautifulSoup(resp.text, "html.parser")
        return soup.get_text(separator="\n", strip=True)
    except Exception as e:
        return f"Error fetching PR guide: {e}"


def check_github_pr(pr_url: str):
    """Call the GitHub API to inspect a PR for common submission issues."""
    try:
        # Expect: https://github.com/{owner}/{repo}/pull/{number}
        parts = pr_url.strip("/").split("/")
        owner, repo, pr_number = parts[-4], parts[-3], parts[-1]

        headers = {"Accept": "application/vnd.github+json"}
        gh_token = os.getenv("GITHUB_TOKEN")
        if gh_token:
            headers["Authorization"] = f"Bearer {gh_token}"

        base  = f"https://api.github.com/repos/{owner}/{repo}/pulls/{pr_number}"
        pr    = requests.get(base, headers=headers, timeout=10).json()
        files = requests.get(f"{base}/files", headers=headers, timeout=10).json()

        filenames = [f["filename"] for f in files]
        outside_community = [
            f for f in filenames if "community-contributions" not in f.lower()
        ]

        return json.dumps({
            "title":         pr.get("title"),
            "state":         pr.get("state"),
            "additions":     pr.get("additions"),
            "deletions":     pr.get("deletions"),
            "changed_files": pr.get("changed_files"),
            "all_files":     filenames,
            "outside_community_contributions": outside_community,
            "warnings": {
                "files_outside_community": bool(outside_community),
                "has_deletions":           pr.get("deletions", 0) > 0,
                "too_many_files":          pr.get("changed_files", 0) > 10,
            },
        }, indent=2)
    except Exception as e:
        return f"Error checking PR: {e}"


TOOL_MAP = {
    "get_pr_guide":    lambda **_: get_pr_guide(),
    "check_github_pr": lambda pr_url, **_: check_github_pr(pr_url),
}

TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_pr_guide",
            "description": (
                "Fetch Ed Donner's official PR guide from edwarddonner.com/pr. "
                "Use this when the student needs detailed PR rules or a checklist."
            ),
            "parameters": {"type": "object", "properties": {}, "required": []},
        },
    },
    {
        "type": "function",
        "function": {
            "name": "check_github_pr",
            "description": (
                "Inspect a GitHub Pull Request for common submission issues: "
                "files outside community-contributions, deletions, PR too large, etc."
            ),
            "parameters": {
                "type": "object",
                "properties": {
                    "pr_url": {
                        "type": "string",
                        "description": "Full GitHub PR URL, e.g. https://github.com/ed-donner/llm_engineering/pull/42",
                    }
                },
                "required": ["pr_url"],
            },
        },
    },
]

SYSTEM_PROMPT = """You are a Teaching Assistant for Ed Donner's LLM Engineering bootcamp (Feb 2026 cohort).
Your job is to guide students through submitting their weekly exercises via GitHub Pull Requests.

Follow these official steps when helping a student:

STEP 1 – FORK & CLONE
  Visit Ed Donner's repo and fork it to your personal GitHub account.
  Then clone your fork to your local machine.

STEP 2 – CREATE A BRANCH
  Never commit directly to main.
  Create a branch named exactly: week1_exercise_bootcamp_feb2026_firstNameLastName
  (swap week1 and the name to match the student's details)

STEP 3 – SET UP YOUR FOLDER
  Navigate to WeekX/community-contributions (the one INSIDE the Week folder — ignore the root-level one).
  Create a sub-folder named after yourself inside community-contributions.

STEP 4 – COMPLETE & PLACE YOUR WORK
  Your solution path should be: WeekX/community-contributions/firstNameLastName

STEP 5 – PUSH & OPEN A PR
  Push your branch to your fork, then open a Pull Request against Ed Donner's original repo.
  Tag @hopeogbons for review.

You have two tools available:
- Use get_pr_guide whenever you need Ed Donner's official PR checklist (outputs cleared, no deletions, etc.)
- Use check_github_pr when a student shares a PR URL — analyse it and clearly report any issues found.

Always be patient, encouraging, and guide students one clear step at a time."""


def chat(message, history, model):
    messages = [{"role": "system", "content": SYSTEM_PROMPT}]
    messages += [{"role": h["role"], "content": h["content"]} for h in history]
    messages.append({"role": "user", "content": message})

    # Resolve any tool calls before streaming the final answer
    while True:
        response = openrouter.chat.completions.create(
            model=model,
            messages=messages,
            tools=TOOLS,
            tool_choice="auto",
        )
        choice = response.choices[0]

        if choice.finish_reason == "tool_calls":
            assistant_msg = choice.message
            messages.append(assistant_msg)
            for tc in assistant_msg.tool_calls:
                fn_name = tc.function.name
                fn_args = json.loads(tc.function.arguments)
                result  = TOOL_MAP[fn_name](**fn_args)
                messages.append({
                    "role":         "tool",
                    "tool_call_id": tc.id,
                    "content":      result,
                })
        else:
            break

    # Stream the final answer
    stream = openrouter.chat.completions.create(
        model=model,
        messages=messages,
        stream=True,
    )
    result = ""
    for chunk in stream:
        if chunk.choices[0].finish_reason == "stop":
            break
        result += chunk.choices[0].delta.content or ""
        yield result


# Gradio UI

gr.ChatInterface(
    fn=chat,
    type="messages",
    title="GitHub PR Assistant – LLM Engineering Bootcamp",
    description=(
        "Ask me anything about submitting your weekly exercise via GitHub PR. "
        "Paste your PR URL and I'll review it for you!"
    ),
    additional_inputs=[
        gr.Dropdown(choices=MODELS, value=MODELS[0], label="Model"),
    ],
).launch()